<a href="https://colab.research.google.com/github/Stp155906/CodeSpaceJPLDemo/blob/main/Introduction_to_comet_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comet-ml
This is a short introduction to comet-ml, aka Comet, which will be a useful tool in training and tracking your models in this course. 

Note that you are by no means required to use Comet for this course, but we recommend experimenting with it a bit as we have found it makes debugging models significantly easier, and produces some pretty pictures while your model is running so you can easily stop training if your model does not seem to be learning. 

If you are familiar with Tensorboard (a training visualization framework integreated into Tensorflow), Comet is similar but support mulitple experiments running at the same time and has deeper reporting capabilities than Tensorboard.

Note that you can use Comet with the both the Tensorflow and PyTorch frameworks, as well as any IDE (because the results are displayed on a web browser). 



# Installation and Getting Started:

First, sign up (free) on comet.ml and obtain an API key at https://www.comet.ml

Then, the recommended way to install comet-ml is by using:


```
pip install comet-ml
```
or
```
conda install comet_ml -c comet_ml
```
This should install the latest version (3.0.2)

We encourage you to look at the startup scripts that are provided at www.comet.ml/help/quickstart in order to see how the API works. Below is a sample script using the PyTorch framework that is adapted from the quickstart guide.




In [ ]:
#The Experiment class is the key feature of Comet we will be using, 
#creating an experiment object in your training script will allow you to track 
#your training
from comet_ml import Experiment
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

#sample hyperparameters for a model that we will create 
hyper_params = {
    "sequence_length": 28,
    "input_size": 28,
    "hidden_size": 128,
    "num_layers":2,
    "num_classes": 10,
    "batch_size": 100,
    "num_epochs":2,
    "learning_rate":0.01
}

API_KEY = #insert your api key from your comet-ml account here
PROJ_NAME = #insert your project name here
WORKSPACE = #insert your comet-ml username here
experiment = Experiment(api_key=API_KEY,
                        project_name=PROJ_NAME, workspace=WORKSPACE)

#Log the hyperparameters you selected to your experiment, you should be 
#able to see these hyperparameters when you run your experiment
experiment.log_parameters(hyper_params)

#load the MNIST dataset as a toy dataset 
train_dataset = dsets.MNIST(root='./data/', 
                           train=True,
                           transform = transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data/', 
                           train=False,
                           transform = transforms.ToTensor())

#Create a dataLoader object for training (see intro to PyTorch for more 
#information)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=hyper_params['batch_size'],
                                           shuffle=True)
#Create a dataLoader object for testing
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=hyper_params['batch_size'],
                                          shuffle=False)
#define your model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial states
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))

        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])
        return out

rnn = RNN(hyper_params['input_size'], hyper_params['hidden_size'], 
          hyper_params['num_layers'], hyper_params['num_classes'])

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=hyper_params['learning_rate'])

'''
Train the Model, calling the train loop under 
the experiment.train() allows you to provide a namespace 
for metrics and parameters that you are tracking. 
For example, "accuracy" is reported as train_accuracy in Comet.
'''
with experiment.train():
    for epoch in range(hyper_params['num_epochs']):
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(train_loader):
            images = Variable(images.view(-1, hyper_params['sequence_length'], hyper_params['input_size']))
            labels = Variable(labels)

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = rnn(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Compute train accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.data).sum()

            # Log to Comet.ml
            experiment.log_metric("accuracy", correct / total, step=i)

            if (i + 1) % 100 == 0:
                print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                      % (epoch + 1, hyper_params['num_epochs'], i + 1, len(train_dataset) // hyper_params['batch_size'], loss.item()))

with experiment.test():
    # Test the Model, create the test namespace in your experiment
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images.view(-1, hyper_params['sequence_length'], hyper_params['input_size']))
        outputs = rnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    experiment.log_metric("accuracy", 100 * correct / total)
    print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

#make sure to end the experiment when your code is done running
experiment.end()



SyntaxError: ignored